In [1]:
import pandas as pd
import numpy as np
import geopandas as gp
import os
import maup
from op_verification.reference_data import *
import boto3
from s3_paths import *
import io
pd.set_option("display.max_columns", None)
import warnings
warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)


maup.progress.enabled = True

**Notes:**

This file contains the process completed to disaggregate the 2020 General Election results at the precinct level for Pennsylvania's statewide races to the 2020 block level. 53 of the original Census blocks were adjusted by the state of Pennsylvania to account for incarcerated individuals which led to an additonal 54 GEOIDs based on splits where A, B and C were appended to the end to note splits. Pennsylvania released a file with population reallocated on the updated boundaries and with updated boundaries but no population reallocation. The RDH used the non-reallocated file because while incarcerated individuals fit into Voting Age Population, they did not likely actually vote in the 2020 election. 

Pennsylvania is the only state that adjusted not only population, but also block geometries. The adjusted Pennsylvania file did not include the P0040001 variable for total Voting Age Population. As a result the RDH took the ratio of P0010001 from the state file to P0010001 from the Census file to estimate P0040001 in the state file. 

state_P0040001 = (state_P0010001 / census_P0010001) * census_P0040001

In the original precinct files some precincts were split due to district boundaries and votes. In the case of JEFFERSON DISTRICT I the subset of the precinct, "019-:-JEFFERSON DISTRICT I-(CONG-16)", was smaller than the block and therefore did not get assigned as part of maup. Upon further investigation it became clear that to assign the 16 votes from this precinct they would either need to go in the wrong district and census block, or in the right district but wrong precinct. As a result, the RDH removed this precinct and those votes were cut as well. The 16 votes belong to the column "GCON16RKEL" from the precinct file. 

For more information or any questions, please reach out to info@redistrictingdatahub.org.

# Load in PA file

## block file set up

In [2]:
block_cols = ['FIPS', 'VTD', 'TRACT', 'BLOCK', 'LOGRECNO', 'P0010001', 'COUNTYFP20', 'COUSUBFP20', 'VTDST20', 'TRACTCE20', 'BLOCKCE20',
       'GEOID20', 'NAME20', 'MTFCC20', 'UR20', 'UACE20', 'UATYPE20', 'FUNCSTAT20', 'ALAND20', 'AWATER20', 'INTPTLAT20', 'INTPTLON20',
       'COUNTY_NME', 'MCD_NUMBER', 'MCD_NAME', 'MCD_TYP_NM', 'MCD_TYP_CD','VTD_NAME', 'BLKGRP', 'geometry']

s3_client = boto3.client('s3')
state_block_gdf = gp.read_file(f'zip+{state_block}')[block_cols]
census_block_gdf = gp.read_file(f'zip+{census_block}')[['GEOID20', 'P0010001', 'P0020001', 'P0040001']]

In [3]:
#match geoids
geo_census_only = set(census_block_gdf['GEOID20']) - set(state_block_gdf['GEOID20'])
geo_state_only = set(state_block_gdf['GEOID20']) - set(census_block_gdf['GEOID20'])

print(len(geo_census_only))
print(len(geo_state_only))

53
107


In [4]:
#check state geoids that have A or B and C appended to end if removed match census
assert set(state_block_gdf['GEOID20'][(state_block_gdf['GEOID20'].str.contains('A'))|(state_block_gdf['GEOID20'].str.contains('B'))|
                               (state_block_gdf['GEOID20'].str.contains('C'))].str.slice(stop = -1)) - geo_census_only == set()

#Check that census geos only in census all accounted for by state file with A and B and C appended
assert geo_census_only - set(state_block_gdf['GEOID20'][(state_block_gdf['GEOID20'].str.contains('A'))|(state_block_gdf['GEOID20'].str.contains('B'))|
                                                 (state_block_gdf['GEOID20'].str.contains('C'))].str.slice(stop = -1)) == set()

#Check that appended A B C all accounted for in state set
assert geo_state_only - set(state_block_gdf['GEOID20'][(state_block_gdf['GEOID20'].str.contains('A'))|(state_block_gdf['GEOID20'].str.contains('B'))|
                                                (state_block_gdf['GEOID20'].str.contains('C'))]) == set()
#Check that 
assert set(state_block_gdf['GEOID20'][(state_block_gdf['GEOID20'].str.contains('A'))|(state_block_gdf['GEOID20'].str.contains('B'))|
                               (state_block_gdf['GEOID20'].str.contains('C'))]) - geo_state_only == set()


#Create dictionary between Census GEOID and state - select GEOIDs with A,B,C appended, create column where that is chopped off
state_block_gdf['census_geoid'] = state_block_gdf['GEOID20']

state_block_gdf.loc[(state_block_gdf['GEOID20'].str.contains('A'))|(state_block_gdf['GEOID20'].str.contains('B'))|
                           (state_block_gdf['GEOID20'].str.contains('C')), 'census_geoid'] = state_block_gdf['GEOID20'].str.slice(stop = -1)


assert set(state_block_gdf['census_geoid']) - set(census_block_gdf['GEOID20']) == set()
assert set(census_block_gdf['GEOID20']) - set(state_block_gdf['census_geoid']) == set() 
assert state_block_gdf['census_geoid'].nunique() == census_block_gdf['GEOID20'].nunique()

## apply ratio to get P0040001 variable

In [5]:
#create census geoid to vap dictionary to then apply to state file and then modify numbers where state modified based on ratio from P0010001
census_geoid_to_vap_dict = pd.Series(census_block_gdf['P0040001'].values, index = census_block_gdf['GEOID20']).to_dict()
state_block_gdf['vap_unmod'] = state_block_gdf['census_geoid'].map(census_geoid_to_vap_dict)


state_to_census_geoid_dict = pd.Series(state_block_gdf['census_geoid'].values, index = state_block_gdf['GEOID20']).to_dict()

#Subset to just be the modified blocks to make run time faster
state_subset = state_block_gdf[state_block_gdf['GEOID20'].isin(geo_state_only)]
census_subset = census_block_gdf[census_block_gdf['GEOID20'].isin(geo_census_only)]

#isolate P0010001 modified ratio from census to state file to then apply to P0020001 from census to state file
ratio_gdf = state_subset[['census_geoid','GEOID20', 'P0010001']].merge(census_subset[['GEOID20', 'P0010001', 'P0020001', 'P0040001']], 
                                                                       left_on = 'census_geoid', right_on = 'GEOID20', how = 'outer', indicator = True)

#Check that everything merged
ratio_gdf[ratio_gdf['_merge']!='both']

ratio_gdf['p1_ratio'] = ratio_gdf['P0010001_x']/ratio_gdf['P0010001_y']


#map ratio from ratio gdf into state gdf
state_geoid_to_ratio_dict = pd.Series(ratio_gdf['p1_ratio'].values, index = ratio_gdf['GEOID20_x'])
state_block_gdf['mod_ratio'] = state_block_gdf['GEOID20'].map(state_geoid_to_ratio_dict)
#apply mod
state_block_gdf['VAP_MOD'] = state_block_gdf['vap_unmod']
state_block_gdf.loc[state_block_gdf['GEOID20'].isin(geo_state_only), 'VAP_MOD'] = state_block_gdf['vap_unmod']*state_block_gdf['mod_ratio']

state_block_gdf = state_block_gdf.fillna(0)
ratio_gdf[ratio_gdf['GEOID20_x'].isin(set(state_block_gdf['GEOID20'][state_block_gdf['VAP_MOD'].isna()]))]

,census_geoid,GEOID20_x,P0010001_x,GEOID20_y,P0010001_y,P0020001,P0040001,_merge,p1_ratio


## precinct file set up

In [6]:
cong20 = gp.read_file('./pa_gen_20_erj/pa_gen_20_cong_prec.shp')
sldl20 = gp.read_file('./pa_gen_20_erj/pa_gen_20_sldl_prec.shp')
st_sldu20 = gp.read_file('./pa_gen_20_erj/pa_gen_20_st_sldu_prec.shp')

#check that file prec ids don't match
print(cong20.shape)
print(sldl20.shape)
print(st_sldu20.shape)

(9177, 61)
(9153, 529)
(9150, 93)


## maup assign and set up

In [7]:
#Functions for running maup
def check_valid_rows(block_gdf, precinct_gdf):
    prec_geom = precinct_gdf.geometry
    valid_rows = precinct_gdf[~(prec_geom.isna() | prec_geom.is_empty)]
    print('valid precinct rows: ', valid_rows.shape)

    block_geom = block_gdf.geometry
    block_valid_rows = block_gdf[~(block_geom.isna() | block_geom.is_empty)]
    print('valid block rows: ', block_valid_rows.shape)
    

def fix_buffer(gdf):
    """
    return (GeoDataFrame) with the 'bufer(0) trick' applied
    :gdf: (GeoDataFrame) object
    Can be useful when trying to mitigate 'self-intersection' issues
    """
    buffered = gdf.buffer(0)
    gdf.drop(columns=["geometry"])
    # gdf['geometry'] = gdf.apply(lambda x: x.geometry.buffer(0), axis=1)
    gdf["geometry"] = buffered
    return gdf


def maup_assignment_series(block_gdf, precinct_gdf):
    block_gdf = block_gdf.to_crs(precinct_gdf.crs)
    print('block_gdf shape: ', block_gdf.shape, '\n precinct_gdf shape: ', precinct_gdf.shape)

    block_gdf["maup_assignment"] = maup.assign(
        fix_buffer(block_gdf), fix_buffer(precinct_gdf)
    )
    print(block_gdf[block_gdf['maup_assignment'].isna()].shape[0], " null assignments in maup series")
    return block_gdf


def maup_pre_vote_setup(block_gdf, precinct_gdf, PRECID_block, DISTID_block, DISTID_prec):
    block_gdf = block_gdf.to_crs(precinct_gdf.crs)
    bgdf = maup_assignment_series(block_gdf, precinct_gdf)
    assert bgdf['maup_assignment'].isna().any() == False
    bgdf[PRECID_block] = bgdf['maup_assignment'].map(lambda idx: str(precinct_gdf.loc[idx]["UNIQUE_ID"]))
    bgdf[DISTID_block] = bgdf["maup_assignment"].map(lambda idx: str(precinct_gdf.loc[idx][DISTID_prec]))
    
    return bgdf

    
def maup_assign_labels(block_gdf, precinct_gdf, precinctid_input, distid_input):
    #Assign precinct IDs to block file using assign series
    block_gdf[precinctid_input] = block_gdf["maup_assignment"].map(
        lambda idx: str(precinct_gdf.loc[idx]["UNIQUE_ID"])
    )
    if "CONG_DIST" in precinct_gdf.columns:
        block_gdf[distid_input] = block_gdf["maup_assignment"].map(
            lambda idx: str(precinct_gdf.loc[idx]["CONG_DIST"])
        )
    if "SLDL_DIST" in precinct_gdf.columns:
        block_gdf[distid_input] = block_gdf["maup_assignment"].map(
            lambda idx: str(precinct_gdf.loc[idx]["SLDL_DIST"])
        )
    if "SLDU_DIST" in precinct_gdf.columns:
        block_gdf[distid_input] = block_gdf["maup_assignment"].map(
            lambda idx: str(precinct_gdf.loc[idx]["SLDU_DIST"])
        )
    print("Label assignment complete")

    return block_gdf


def election_cols(gdf):
    election_cols = list(gdf.columns[gdf.columns.str.startswith('P20')|gdf.columns.str.startswith('PS')|gdf.columns.str.startswith('PC')|
                     gdf.columns.str.startswith('G20')|gdf.columns.str.startswith('GS')|gdf.columns.str.startswith('GC')])
    return election_cols  


def assign_votes(variables, election_columns, precinct_gdf, block_gdf):
    precinct_gdf[variables] = block_gdf[variables].groupby(block_gdf["maup_assignment"]).sum()
    print(variables, ' added to precinct_gdf based on maup assignment')
    bl_to_prec_weights = block_gdf[variables] / block_gdf["maup_assignment"].map(precinct_gdf[variables])
    block_votes = block_gdf.copy(deep=True)
    block_votes[election_columns] = maup.prorate(
        block_gdf["maup_assignment"], precinct_gdf[election_columns], bl_to_prec_weights
    )
    return block_votes


def run_maup(block_gdf, precinct_gdf, precinctid_input, distid_input, pop_variable):
    #Assign identifiers
    block_gdf = maup_assignment_series(block_gdf, precinct_gdf)
    block_gdf = maup_assign_labels(block_gdf, precinct_gdf, precinctid_input, distid_input)

    #Assign votes
    election_columns = election_cols(precinct_gdf)
    block_votes = assign_votes(pop_variable, election_columns, precinct_gdf, block_gdf)
    
    return block_votes    



In [13]:
def column_total_check(election_columns, block_gdf, precinct_gdf):
    for val in election_columns:
        vote_dif = block_gdf[val].sum()-precinct_gdf[val].sum()
        if (abs(vote_dif) <=1e-1):
            #print(val+": EQUAL", ' - total: ', 'block:', str(block_gdf[val].sum()), 'prec:', str(precinct_gdf[val].sum()), 'diff:', block_gdf[val].sum()-precinct_gdf[val].sum())
            continue
        else:
            print(val+": DIFFERENCE OF " + str(vote_dif)+ " VOTES", ' - block total: ', str(block_gdf[val].sum()), ', precinct total: ', str(precinct_gdf[val].sum()))     
            

set_in_prec_only = {'003-:-SOUTH FAYETTE DISTRICT 05-(CONG-14)',
 '019-:-JEFFERSON DISTRICT I-(CONG-16)',
 '027-:-HALFMOON PRECINCT PROPER-(CONG-13)',
 '041-:-NORTH MIDDLETON PRECINCT 01-(CONG-12)',
 '041-:-NORTH MIDDLETON PRECINCT 03-(CONG-12)'}
cong20 = cong20[~cong20['UNIQUE_ID'].isin(set_in_prec_only)]

cong20.crs = "EPSG:4269"
bcong = maup_pre_vote_setup(state_block_gdf, cong20, "PREC_CONG", "CONG_DIST", "CONG_DIST")

#come back to modify to use P0040001 estimate when ready!
bcong_votes = assign_votes("VAP_MOD", election_cols(cong20), cong20, bcong)

try:
    column_total_check(election_cols(cong20), bcong, cong20)
except:
    print('column check failed, come back to')
    
print("cong disag complete")


sldl20.crs = "EPSG:4269"
bsldl = maup_pre_vote_setup(state_block_gdf, sldl20, "PREC_SLDL", "SLDL_DIST", "SLDL_DIST")

#come back to modify to use P0040001 estimate when ready!
bsldl_votes = assign_votes("VAP_MOD", election_cols(sldl20), sldl20, bsldl)

try:
    column_total_check(election_cols(sldl20), bsldl, sldl20)
except:
    print('column check failed, come back to')
    
print("sldl disag complete")


st_sldu20.crs = "EPSG:4269"
bst_sldu = maup_pre_vote_setup(state_block_gdf, st_sldu20, "PREC_ST_SLDU", "SLDU_DIST", "SLDU_DIST")

#come back to modify to use P0040001 estimate when ready!
bst_sldu_votes = assign_votes("VAP_MOD", election_cols(st_sldu20), st_sldu20, bst_sldu)

try:
    column_total_check(election_cols(st_sldu20), bst_sldu, st_sldu20)
except:
    print('column check failed, come back to')
    
print("st/sldu disag complete")

block_gdf shape:  (337039, 34) 
 precinct_gdf shape:  (9172, 62)


100%|██████████| 9172/9172 [01:14<00:00, 122.90it/s]
/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/maup/intersections.py:42: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/maup/assign.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, area_cutoff=0).area)


0  null assignments in maup series
VAP_MOD  added to precinct_gdf based on maup assignment
column check failed, come back to
cong disag complete
block_gdf shape:  (337039, 34) 
 precinct_gdf shape:  (9153, 530)


100%|██████████| 9153/9153 [01:14<00:00, 122.86it/s]
/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/maup/intersections.py:42: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/maup/assign.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, area_cutoff=0).area)


0  null assignments in maup series
VAP_MOD  added to precinct_gdf based on maup assignment
column check failed, come back to
sldl disag complete
block_gdf shape:  (337039, 34) 
 precinct_gdf shape:  (9150, 94)


100%|██████████| 9150/9150 [01:11<00:00, 128.40it/s]
/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/maup/intersections.py:42: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/maup/assign.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, area_cutoff=0).area)


0  null assignments in maup series
VAP_MOD  added to precinct_gdf based on maup assignment
column check failed, come back to
st/sldu disag complete


## Merge

In [14]:
b_merge = bst_sldu_votes.merge(bsldl_votes, 
             how='outer', 
             on=["GEOID20", "FIPS", "TRACT", "BLOCKCE20", "NAME20", "ALAND20", "AWATER20", 
                 "geometry"], 
             suffixes=('_sldu','_sldl')
            ).merge(bcong_votes, how='outer', 
                    on=["GEOID20", "FIPS", "TRACT", "BLOCKCE20", "NAME20", "ALAND20", "AWATER20", 
                        "geometry"], suffixes=('_sldl', '_cong')
                   )

In [15]:
b_merge.head()

,FIPS,VTD_sldu,TRACT,BLOCK_sldu,LOGRECNO_sldu,P0010001_sldu,COUNTYFP20_sldu,COUSUBFP20_sldu,VTDST20_sldu,TRACTCE20_sldu,BLOCKCE20,GEOID20,NAME20,MTFCC20_sldu,UR20_sldu,UACE20_sldu,UATYPE20_sldu,FUNCSTAT20_sldu,ALAND20,AWATER20,INTPTLAT20_sldu,INTPTLON20_sldu,COUNTY_NME_sldu,MCD_NUMBER_sldu,MCD_NAME_sldu,MCD_TYP_NM_sldu,MCD_TYP_CD_sldu,VTD_NAME_sldu,BLKGRP_sldu,geometry,census_geoid_sldu,vap_unmod_sldu,mod_ratio_sldu,VAP_MOD_sldu,maup_assignment_sldu,PREC_ST_SLDU,SLDU_DIST,G20ATGDSHA,G20ATGGWEI,G20ATGLWAS,G20ATGOWRI,G20ATGRHEI,G20AUDDAHM,G20AUDGFAI,G20AUDLMOO,G20AUDOWRI,G20AUDRDEF,G20PREDBID,G20PREGHAW,G20PRELJOR,G20PREOWRI,G20PRERTRU,G20TREDTOR,G20TREGRUN,G20TRELSOL,G20TREOWRI,G20TRERGAR,GSU01DSAV,GSU01OWRI,GSU03DSTR,GSU03OWRI,GSU05DSAB,GSU05OWRI,GSU07DHUG,GSU07OWRI,GSU09DKAN,GSU09RKIL,GSU11DSCH,GSU11OWRI,GSU11RBAK,GSU13DDIA,GSU13OWRI,GSU13RMAR,GSU15DSCO,GSU15OWRI,GSU15RDIS,GSU17DCAP,GSU17OWRI,GSU17RFIS,GSU19DCOM,GSU19RRUN,GSU21DSTR,GSU21OWRI,GSU21RHUT,GSU23DBAK,GSU23OWRI,GSU23RYAW,GSU25DBRO,GSU25OWRI,GSU25RDUS,GSU27DSIE,GSU27OWRI,GSU27RGOR,GSU29OWRI,GSU29RARG,GSU31DDAN,GSU31OWRI,GSU31RREG,GSU33DSTE,GSU33OWRI,GSU33RMAS,GSU35DDOU,GSU35OWRI,GSU35RLAN,GSU37DIOV,GSU37OWRI,GSU37RROB,GSU39DWAL,GSU39OWRI,GSU39RWAR,GSU41DDEL,GSU41OWRI,GSU41RPIT,GSU43DCOS,GSU43OWRI,GSU45DBRE,GSU45OWRI,GSU45RZIC,GSU47DKRI,GSU47OWRI,GSU47RVOG,GSU49DSLO,GSU49RLAU,VTD_sldl,BLOCK_sldl,LOGRECNO_sldl,P0010001_sldl,COUNTYFP20_sldl,COUSUBFP20_sldl,VTDST20_sldl,TRACTCE20_sldl,MTFCC20_sldl,UR20_sldl,UACE20_sldl,UATYPE20_sldl,FUNCSTAT20_sldl,INTPTLAT20_sldl,INTPTLON20_sldl,COUNTY_NME_sldl,MCD_NUMBER_sldl,MCD_NAME_sldl,MCD_TYP_NM_sldl,MCD_TYP_CD_sldl,VTD_NAME_sldl,BLKGRP_sldl,census_geoid_sldl,vap_unmod_sldl,mod_ratio_sldl,VAP_MOD_sldl,maup_assignment_sldl,PREC_SLDL,SLDL_DIST,GSL001DHAR,GSL002DMER,GSL003DBIZ,GSL003RHAY,GSL004RSON,GSL005DGON,GSL005OWRI,GSL005RJOZ,GSL006DFER,GSL006RROA,GSL007DLON,GSL007OWRI,GSL008DHEA,GSL008OWRI,GSL008RBON,GSL009DSAI,GSL009IAUD,GSL009OWRI,GSL009RRYA,GSL010DCOL,GSL010OWRI,GSL010RBER,GSL010UPEF,GSL011DDOC,GSL011OWRI,GSL011RMUS,GSL012DSMI,GSL012OWRI,GSL012RMET,GSL013DRUG,GSL013OWRI,GSL013RLAW,GSL014DWIL,GSL014OWRI,GSL014RMAR,GSL015DWIL,GSL015OWRI,GSL015RKAI,GSL016DMAT,GSL016OWRI,GSL016RELM,GSL017OWRI,GSL017RWEN,GSL018DHAY,GSL018OWRI,GSL018RTOM,GSL019DWHE,GSL019OWRI,GSL020DKIN,GSL020OWRI,GSL021DINN,GSL021OWRI,GSL021RWAU,GSL022DSCH,GSL023DFRA,GSL023GWAL,GSL023OWRI,GSL024DGAI,GSL024OWRI,GSL025DMAR,GSL025OWRI,GSL025RRIT,GSL026DFRI,GSL026OWRI,GSL026RHEN,GSL027DDEA,GSL027OWRI,GSL028DSKO,GSL028OWRI,GSL028RMER,GSL029ASCH,GSL029DKAT,GSL029OWRI,GSL029RSCH,GSL030DSHU,GSL030OWRI,GSL030RMIZ,GSL031DWAR,GSL031OWRI,GSL031RADC,GSL032DDEL,GSL032OWRI,GSL033DDER,GSL033OWRI,GSL033RDEL,GSL034DLEE,GSL034OWRI,GSL035DDAV,GSL035OWRI,GSL036DBEN,GSL036OWRI,GSL036RDOY,GSL037DPAD,GSL037OWRI,GSL037RFEE,GSL038DPIS,GSL038OWRI,GSL038RBOO,GSL039DOLI,GSL039OWRI,GSL039RPUS,GSL040DGUI,GSL040OWRI,GSL040RMIH,GSL041DWHE,GSL041OWRI,GSL041RMIL,GSL042DMIL,GSL042OWRI,GSL042RKOR,GSL043OWRI,GSL043RGRE,GSL044DKNO,GSL044OWRI,GSL044RGAY,GSL045DKUL,GSL045OWRI,GSL045RDEV,GSL046DTIM,GSL046OWRI,GSL046RORT,GSL047DOWE,GSL047OWRI,GSL047RGIL,GSL048DSHO,GSL048RONE,GSL049DBAR,GSL049RCOO,GSL050DSNY,GSL050OWRI,GSL050RYOS,GSL051DJON,GSL051OWRI,GSL051RDOW,GSL052DCOC,GSL052OWRI,GSL052RWAR,GSL053DMAL,GSL053OWRI,GSL053RARN,GSL054OWRI,GSL054RBRO,GSL055DPET,GSL055OWRI,GSL055RSIL,GSL056OWRI,GSL056RDUN,GSL057OWRI,GSL057RNEL,GSL058DPRA,GSL058OWRI,GSL058RDAV,GSL059OWRI,GSL059RREE,GSL060OWRI,GSL060RPYL,GSL061DHAN,GSL061OWRI,GSL061RFRI,GSL062DSEM,GSL062OWRI,GSL062RSTR,GSL063OWRI,GSL063ROBE,GSL064GBAG,GSL064OWRI,GSL064RJAM,GSL065OWRI,GSL065RRAP,GSL066OWRI,GSL066RSMI,GSL067OWRI,GSL067RCAU,GSL068LLAW,GSL068OWRI,GSL068ROWL,GSL069OWRI,GSL069RMET,GSL070DBRA,GSL070OWRI,GSL070RWIS,GSL071OWRI,GSL071RRIG,GSL072DBUR,GSL072OWRI,GSL072RTER,GSL073OWRI,GSL073RSAN,GSL074DWIL,GSL074RHEN,GSL075DGRI,GSL075OWRI,GSL075RARM,GSL076DWAL,GSL076OWRI,GSL076RBOR,GSL077DCON,GSL077OWRI,GSL077RYET,GSL078OWRI,GSL078RTOP,GSL079DRUN,GSL079OWRI

In [16]:
assert state_block_gdf.shape[0]==b_merge.shape[0]

In [17]:
b_merge['VAP_MOD']

0          6.000000
1          0.000000
2          4.000000
3         11.000000
4         69.000000
            ...    
337034     0.000000
337035     0.000000
337036    28.465116
337037    23.625000
337038     8.742775
Name: VAP_MOD, Length: 337039, dtype: float64

# Checks

In [18]:
def check_block_to_prec_pop_prop_sum(pop_col_list, VAP_MOD_COL, PREC_COL):
    #TODO change pop cols checking
    pop_col_list = pop_col_list + ['GEOID20', 'P0010001_sldu']
    popcd = b_merge[pop_col_list]
    prec_popcd = popcd.groupby(PREC_COL).sum().reset_index()

    prec_to_pop_dict_gcd = pd.Series(prec_popcd[VAP_MOD_COL].values, index = prec_popcd[PREC_COL]).to_dict()
    
    #print(prec_to_pop_dict_gcd)
    
    popcd['prec_pop'] = popcd[PREC_COL].map(prec_to_pop_dict_gcd)
    popcd['prop'] = popcd[VAP_MOD_COL]/popcd['prec_pop']
    
    print(popcd.groupby(PREC_COL).sum()['prop'].sum())
    print(popcd[PREC_COL].nunique())
    

print(check_block_to_prec_pop_prop_sum(['PREC_CONG', 'VAP_MOD'], 'VAP_MOD', 'PREC_CONG'))
print(check_block_to_prec_pop_prop_sum(['PREC_ST_SLDU', 'VAP_MOD'], 'VAP_MOD', 'PREC_ST_SLDU'))
print(check_block_to_prec_pop_prop_sum(['PREC_SLDL', 'VAP_MOD'], 'VAP_MOD', 'PREC_SLDL'))

<ipython-input-18-5bf6a63aa161>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popcd['prec_pop'] = popcd[PREC_COL].map(prec_to_pop_dict_gcd)
<ipython-input-18-5bf6a63aa161>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popcd['prop'] = popcd[VAP_MOD_COL]/popcd['prec_pop']


9171.0
9172
None
9149.0
9150
None
9152.0
9153
None


Prec #s
(9177, 61)
(9153, 529)
(9150, 93)

In [19]:
#Column total checks
column_total_check(election_cols(cong20), b_merge, cong20)
column_total_check(election_cols(st_sldu20), b_merge, st_sldu20)
column_total_check(election_cols(sldl20), b_merge, sldl20)

In [23]:
def precinct_sum_check(prec_gdf, block_gdf, blk_prec_id):
    prec_gpbyprec = prec_gdf.groupby(['UNIQUE_ID']).sum()
    blk_gpbyprec = block_gdf.groupby([blk_prec_id]).sum()
    prec_w_difs_list = []
    
    prec_gpbyprec.sort_values(by='UNIQUE_ID',inplace=True)
    prec_gpbyprec.reset_index(inplace = True)
    blk_gpbyprec.sort_values(by= blk_prec_id, inplace=True)
    blk_gpbyprec.reset_index(inplace = True)
    
    for i in range(0, prec_gdf['UNIQUE_ID'].nunique()):
        for col in election_cols(prec_gdf):
            prec_diff = round(blk_gpbyprec[col].iloc[i], 1) - round(prec_gpbyprec[col].iloc[i], 1)
            if (prec_diff!=0):
                prec_w_difs_list.append(prec_gpbyprec.index[i])
                print("prec id: ", blk_gpbyprec.index[i], prec_gpbyprec.index[i], col, "block: ", round(blk_gpbyprec[col].iloc[i],1), 
                      "prec: ", round(prec_gpbyprec[col].iloc[i],1), "diff: ", prec_diff)
    print("precs to check: ", set(prec_w_difs_list))
    print("Precinct check complete!")


#Precinct total checks
precinct_sum_check(cong20, b_merge, "PREC_CONG")
precinct_sum_check(sldl20, b_merge, "PREC_SLDL")
precinct_sum_check(st_sldu20, b_merge, "PREC_ST_SLDU")

precs to check:  set()
Precinct check complete!
precs to check:  set()
Precinct check complete!
precs to check:  set()
Precinct check complete!


In [ ]:
#Check for column name length

## Export

In [110]:
def format_gdf(block_votes, election_columns):
    #block_votes['VAP'] = block_votes['P0040001']
    ordered_cols = ['GEOID20', 'NAME20', 'STATEFP20', 'COUNTYFP20', 'TRACTCE20', 'BLOCKCE20', 'PRECINCTID',
                    'CONG_DIST', 'SLDL_DIST', 'SLDU_DIST', 'VAP_MOD'] + \
                   election_columns + \
                   ['ALAND20', 'AWATER20', 'geometry']
    block_votes_export = block_votes[ordered_cols].fillna(0)
    return block_votes_export


def create_shp(gdf, shp_name):
    os.mkdir('./' + shp_name)
    gdf.to_file('./' + shp_name + '/' + shp_name + '.shp')
    print(shp_name, 'shapefile created.')

In [46]:
b_merge['PRECINCTID'] = b_merge['PREC_ST_SLDU']
b_merge['STATEFP20'] = "42"
b_export = format_gdf(b_merge, election_cols(b_merge))

In [111]:
b_export = format_gdf(b_merge, election_cols(b_merge))
create_shp(b_export, "pa_20_block")

pa_20_block shapefile created.


## Readme

In [92]:
def export_readme(readme_name, full_readme_text):
    ''':return: README to txt file from readme string
    :param: readme_name: file path to readme ex: './az_gen_20_prec/README.txt'
    '''
    with open(readme_name, 'x') as tf:
        tf.write(full_readme_text)
    print(readme_name, " readme moved to folder")

In [78]:
fields_2_idx = b_export.columns[b_export.columns.isin(pa_disag_fields_dict.keys())]
fields_2_vals = fields_2_idx.map(pa_disag_fields_dict)
fields_dict_2 = pd.Series(fields_2_vals.values, index = fields_2_idx).to_dict()
fields_dict_2

{'G20ATGDSHA': 'Josh Shapiro Attorney General (DEM)',
 'G20ATGGWEI': 'Richard Weiss Attorney General (GRN)',
 'G20ATGLWAS': 'Daniel Wassmer Attorney General (LIB)',
 'G20ATGOWRI': 'Write-In Attorney General (N/A)',
 'G20ATGRHEI': 'Heather Heidelbaugh Attorney General (REP)',
 'G20AUDDAHM': 'Nina Ahmad Auditor General (DEM)',
 'G20AUDGFAI': 'Olivia Faison Auditor General (GRN)',
 'G20AUDLMOO': 'Jennifer Moore Auditor General (LIB)',
 'G20AUDOWRI': 'Write-In Auditor General (N/A)',
 'G20AUDRDEF': 'Timothy Defoor Auditor General (REP)',
 'G20PREDBID': 'Joseph R. Biden President (DEM)',
 'G20PREGHAW': 'Howie Hawkins President (GRN)',
 'G20PRELJOR': 'Jo Jorgensen President (LIB)',
 'G20PREOWRI': 'Other / Write-In President (N/A)',
 'G20PRERTRU': 'Donald J. Trump President (REP)',
 'G20TREDTOR': 'Joe Torsella State Treasurer (DEM)',
 'G20TREGRUN': 'Timothy Runkle State Treasurer (GRN)',
 'G20TRELSOL': 'Joe Soloski State Treasurer (LIB)',
 'G20TREOWRI': 'Write-In State Treasurer (N/A)',
 'G20

In [94]:
def create_fields_table(race_field_header_0, fields_dict_0, 
                        add_race_field_header_1 = '', fields_dict_1 = {}, 
                        add_race_field_header_2 = '', fields_dict_2 = {}, 
                        add_race_field_header_3 = '', fields_dict_3 = {}):
    fields_table_data = {'Field Name': ['',race_field_header_0]  + list(fields_dict_0.keys()) +
                         ['',add_race_field_header_1] + list(fields_dict_1.keys()) +
                         ['',add_race_field_header_2] + list(fields_dict_2.keys()) +
                         ['',add_race_field_header_3] + list(fields_dict_3.keys()),
                         'Description': ['',''] + list(fields_dict_0.values()) + 
                         ['',''] + list(fields_dict_1.values()) + 
                         ['',''] + list(fields_dict_2.values()) +
                         ['',''] + list(fields_dict_3.values())}
    fields_table = pd.DataFrame(fields_table_data)
    return fields_table




fields_dict_0 = {'GEOID20': 'Geographic Record Identifier from the 2020 Census',
                 'NAME20': 'Area Name-Legal/Statistical, Area Description (LSAD), Term-Part Indicator',
                 'STATEFP20': '2020 State (FIPS)',
                 'COUNTYFP20': '2020 County (FIPS)',
                 'TRACTCE20': '2020 Census Tract',
                 'BLOCKCE20': '2020 Census Block',
                 'PRECINCTID': 'Unique precinct name/identifier',
                 'CONG_DIST': 'US Representative district number',
                 'SLDL_DIST': 'State Legislative Lower Level district number',
                 'SLDU_DIST': 'State Legislative Upper Level district number',
                 'VAP_MOD': 'Estimated Total Voting Age Population - see additional notes below'
                 }
fields_dict_1 = {' ': ' '}

fields_dict_3 = {'ALAND20': 'Area (Land)',
                 'AWATER20': 'Area (Water)',
                 'geometry': 'geometry'
                 }
create_fields_table('', fields_dict_0, '',fields_dict_1, '',fields_dict_2, '', fields_dict_3)


def disag_readme_template(stateabrv, state, year, election_type, retrieval_date,
                        erj_file_source, block_file_source, office_codes, 
                        race_field_header_0, fields_dict_0, add_race_field_header_1 = '', fields_dict_1 = {}, add_race_field_header_2 = '', fields_dict_2 = {}, 
                        add_race_field_header_3 = '', fields_dict_3 = {},
                       additional_notes=' '):

#First section of README
    readme_p1 = '''{year} {stateabrv} {election_type} Disaggregated Election Results

## RDH Date Retrieval
{retrieval_date}

## Sources
The RDH retrieved raw {year} {election_type} election results from {raw_data_source}
The RDH retrieved the PL Census Block shapefile from {block_file_source}

## Notes on Field Names (adapted from VEST):
Columns reporting votes generally follow the pattern: 
The first character is G for a general election, P for a primary, S for a special, and R for a runoff.
Characters 2 and 3 are the year of the election.*
Characters 4-6 represent the office type (see list below).
Character 7 represents the party of the candidate.
Characters 8-10 are the first three letters of the candidate's last name.
One example is:
G16PREDCLI

To fit within the GIS 10 character limit for field names, the naming convention is slightly different for the State Legislature and 
US House of Representatives. All fields are listed below with definitions.

Office Codes Used:
{office_codes}

## Fields:

'''.format(stateabrv= stateabrv, state= state, year=year, election_type=election_type,retrieval_date=retrieval_date, raw_data_source=erj_file_source, office_codes=office_codes, block_file_source = block_file_source)

#Second section of README
    fields_table = create_fields_table(race_field_header_0, fields_dict_0, add_race_field_header_1, fields_dict_1, add_race_field_header_2, fields_dict_2, 
                        add_race_field_header_3, fields_dict_3)
    readme_p2 = fields_table.to_string(formatters={'Description':'{{:<{}s}}'.format(fields_table['Description'].str.len().max()).format, 'Field Name':'{{:<{}s}}'.format(fields_table['Field Name'].str.len().max()).format}, index=False)

#Third section of README
    readme_p3 = '''
## Processing Steps
    
The RDH used Python to disaggregate precinct election results to the Census Block level. For additional details, visit our Github https://github.com/nonpartisan-redistricting-datahub/.

The RDH validated the disaggregated election results against the state and county totals in the original precinct election results file. The precinct level results were checked during our ERJ process. For more information on the ERJ project and validation, please see our processing on Github.

## Additional Notes
{additional_notes}

Please contact info@redistrictingdatahub.org for more information.
'''.format(stateabrv=stateabrv, state=state, year=year, election_type=election_type, office_codes=office_codes, additional_notes = additional_notes)
    
    full_readme = str(readme_p1)+str(readme_p2)+str(readme_p3)
    return full_readme



# Export README.txt
office_codes = {'PRE': 'President of the United States',
                     'CONXX': 'U.S. House, accompanied by a CON_DIST column indicating district number',
                'SUXX': 'State Legislative Upper District, accompanied by a SLDU_DIST column indicating district number',
                'SLXX': 'State Legislative Lower District, accompanied by a SLDL_DIST column indicating district number'
                }
stateabrv = 'PA'
state = 'Pennsylvania'
year = '2020'
election_type = 'General'
retrieval_date = '08/22/2022'
erj_file_source = 'https://redistrictingdatahub.org/dataset/pennsylvania-2020-general-election-precinct-and-election-results-extended/'
block_file_source = 'https://redistrictingdatahub.org/dataset/pennsylvania-block-pl-94-171-2020-official-without-prisoner-reallocation/'
additional_notes = '''This file contains the process completed to disaggregate the 2020 General Election results at the precinct level for Pennsylvania's statewide races to the 2020 block level. 53 of the original Census blocks were adjusted by the state of Pennsylvania to account for incarcerated individuals which led to an additonal 54 GEOIDs based on splits where A, B and C were appended to the end to note splits. Pennsylvania released a file with population reallocated on the updated boundaries and with updated boundaries but no population reallocation. The RDH used the non-reallocated file because while incarcerated individuals fit into Voting Age Population, they did not likely actually vote in the 2020 election. 

Pennsylvania is the only state that adjusted not only population, but also block geometries. The adjusted Pennsylvania file did not include the P0040001 variable for total Voting Age Population. As a result the RDH took the ratio of P0010001 from the state file to P0010001 from the Census file to estimate P0040001 in the state file. 

state_P0040001 = (state_P0010001 / census_P0010001) * census_P0040001

In the original precinct files some precincts were split due to district boundaries and votes. In the case of JEFFERSON DISTRICT I the subset of the precinct, "019-:-JEFFERSON DISTRICT I-(CONG-16)", was smaller than the block and therefore did not get assigned as part of maup. Upon further investigation it became clear that to assign the 16 votes from this precinct they would either need to go in the wrong district and census block, or in the right district but wrong precinct. As a result, the RDH removed this precinct and those votes were cut as well. The 16 v'''

pa_readme_text = disag_readme_template(stateabrv, state, year, election_type, retrieval_date,
                        erj_file_source, block_file_source, office_codes, 
                        '', fields_dict_0, '',fields_dict_1, '',fields_dict_2, '', fields_dict_3,
                       additional_notes=' ')


export_readme("pa_20_block/README.txt", pa_readme_text)

pa_20_block/README.txt  readme moved to folder


In [95]:
b_export.columns[b_export.isna()]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [96]:
b_export[b_export.isna()]

,GEOID20,NAME20,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,PRECINCTID,CONG_DIST,SLDL_DIST,SLDU_DIST,VAP_MOD,G20ATGDSHA,G20ATGGWEI,G20ATGLWAS,G20ATGOWRI,G20ATGRHEI,G20AUDDAHM,G20AUDGFAI,G20AUDLMOO,G20AUDOWRI,G20AUDRDEF,G20PREDBID,G20PREGHAW,G20PRELJOR,G20PREOWRI,G20PRERTRU,G20TREDTOR,G20TREGRUN,G20TRELSOL,G20TREOWRI,G20TRERGAR,GSU01DSAV,GSU01OWRI,GSU03DSTR,GSU03OWRI,GSU05DSAB,GSU05OWRI,GSU07DHUG,GSU07OWRI,GSU09DKAN,GSU09RKIL,GSU11DSCH,GSU11OWRI,GSU11RBAK,GSU13DDIA,GSU13OWRI,GSU13RMAR,GSU15DSCO,GSU15OWRI,GSU15RDIS,GSU17DCAP,GSU17OWRI,GSU17RFIS,GSU19DCOM,GSU19RRUN,GSU21DSTR,GSU21OWRI,GSU21RHUT,GSU23DBAK,GSU23OWRI,GSU23RYAW,GSU25DBRO,GSU25OWRI,GSU25RDUS,GSU27DSIE,GSU27OWRI,GSU27RGOR,GSU29OWRI,GSU29RARG,GSU31DDAN,GSU31OWRI,GSU31RREG,GSU33DSTE,GSU33OWRI,GSU33RMAS,GSU35DDOU,GSU35OWRI,GSU35RLAN,GSU37DIOV,GSU37OWRI,GSU37RROB,GSU39DWAL,GSU39OWRI,GSU39RWAR,GSU41DDEL,GSU41OWRI,GSU41RPIT,GSU43DCOS,GSU43OWRI,GSU45DBRE,GSU45OWRI,GSU45RZIC,GSU47DKRI,GSU47OWRI,GSU47RVOG,GSU49DSLO,GSU49RLAU,GSL001DHAR,GSL002DMER,GSL003DBIZ,GSL003RHAY,GSL004RSON,GSL005DGON,GSL005OWRI,GSL005RJOZ,GSL006DFER,GSL006RROA,GSL007DLON,GSL007OWRI,GSL008DHEA,GSL008OWRI,GSL008RBON,GSL009DSAI,GSL009IAUD,GSL009OWRI,GSL009RRYA,GSL010DCOL,GSL010OWRI,GSL010RBER,GSL010UPEF,GSL011DDOC,GSL011OWRI,GSL011RMUS,GSL012DSMI,GSL012OWRI,GSL012RMET,GSL013DRUG,GSL013OWRI,GSL013RLAW,GSL014DWIL,GSL014OWRI,GSL014RMAR,GSL015DWIL,GSL015OWRI,GSL015RKAI,GSL016DMAT,GSL016OWRI,GSL016RELM,GSL017OWRI,GSL017RWEN,GSL018DHAY,GSL018OWRI,GSL018RTOM,GSL019DWHE,GSL019OWRI,GSL020DKIN,GSL020OWRI,GSL021DINN,GSL021OWRI,GSL021RWAU,GSL022DSCH,GSL023DFRA,GSL023GWAL,GSL023OWRI,GSL024DGAI,GSL024OWRI,GSL025DMAR,GSL025OWRI,GSL025RRIT,GSL026DFRI,GSL026OWRI,GSL026RHEN,GSL027DDEA,GSL027OWRI,GSL028DSKO,GSL028OWRI,GSL028RMER,GSL029ASCH,GSL029DKAT,GSL029OWRI,GSL029RSCH,GSL030DSHU,GSL030OWRI,GSL030RMIZ,GSL031DWAR,GSL031OWRI,GSL031RADC,GSL032DDEL,GSL032OWRI,GSL033DDER,GSL033OWRI,GSL033RDEL,GSL034DLEE,GSL034OWRI,GSL035DDAV,GSL035OWRI,GSL036DBEN,GSL036OWRI,GSL036RDOY,GSL037DPAD,GSL037OWRI,GSL037RFEE,GSL038DPIS,GSL038OWRI,GSL038RBOO,GSL039DOLI,GSL039OWRI,GSL039RPUS,GSL040DGUI,GSL040OWRI,GSL040RMIH,GSL041DWHE,GSL041OWRI,GSL041RMIL,GSL042DMIL,GSL042OWRI,GSL042RKOR,GSL043OWRI,GSL043RGRE,GSL044DKNO,GSL044OWRI,GSL044RGAY,GSL045DKUL,GSL045OWRI,GSL045RDEV,GSL046DTIM,GSL046OWRI,GSL046RORT,GSL047DOWE,GSL047OWRI,GSL047RGIL,GSL048DSHO,GSL048RONE,GSL049DBAR,GSL049RCOO,GSL050DSNY,GSL050OWRI,GSL050RYOS,GSL051DJON,GSL051OWRI,GSL051RDOW,GSL052DCOC,GSL052OWRI,GSL052RWAR,GSL053DMAL,GSL053OWRI,GSL053RARN,GSL054OWRI,GSL054RBRO,GSL055DPET,GSL055OWRI,GSL055RSIL,GSL056OWRI,GSL056RDUN,GSL057OWRI,GSL057RNEL,GSL058DPRA,GSL058OWRI,GSL058RDAV,GSL059OWRI,GSL059RREE,GSL060OWRI,GSL060RPYL,GSL061DHAN,GSL061OWRI,GSL061RFRI,GSL062DSEM,GSL062OWRI,GSL062RSTR,GSL063OWRI,GSL063ROBE,GSL064GBAG,GSL064OWRI,GSL064RJAM,GSL065OWRI,GSL065RRAP,GSL066OWRI,GSL066RSMI,GSL067OWRI,GSL067RCAU,GSL068LLAW,GSL068OWRI,GSL068ROWL,GSL069OWRI,GSL069RMET,GSL070DBRA,GSL070OWRI,GSL070RWIS,GSL071OWRI,GSL071RRIG,GSL072DBUR,GSL072OWRI,GSL072RTER,GSL073OWRI,GSL073RSAN,GSL074DWIL,GSL074RHEN,GSL075DGRI,GSL075OWRI,GSL075RARM,GSL076DWAL,GSL076OWRI,GSL076RBOR,GSL077DCON,GSL077OWRI,GSL077RYET,GSL078OWRI,GSL078RTOP,GSL079DRUN,GSL079OWRI,GSL079RSCH,GSL080OWRI,GSL080RGRE,GSL081DKID,GSL081OWRI,GSL081RIRV,GSL082OWRI,GSL082RHER,GSL083DPAG,GSL083OWRI,GSL083RWHE,GSL084DWAL,GSL084OWRI,GSL084RHAM,GSL085DEVA,GSL085OWRI,GSL085RROW,GSL086OWRI,GSL086RSTA,GSL087DMIL,GSL087OWRI,GSL087RROT,GSL088DSHA,GSL088OWRI,GSL088RDEL,GSL089OWRI,GSL089RKAU,GSL090OWRI,GSL090RSCH,GSL091OWRI,GSL091RMOU,GSL092DROS,GSL092OWRI,GSL092RKEE,GSL093OWRI,GSL093RJON,GSL094OWRI,GSL094RSAY,GSL095DHIL,GSL095OWRI,GSL095RFRE,GSL096DSTU,GSL096OWRI,GSL097DGUL,GSL097OWRI,GSL097RMEN,GSL098DTRO,GSL098OWRI,GSL098RHIC,GSL099DHOD,GSL099OWRI,GSL099RZIM,GSL100OWRI,GSL100RCUT,GSL101DCLE,GSL101OWRI,GSL101RRYA,GSL102DDUV,GSL102OWRI,GSL102RDIA,GSL103DKIM,GSL103OWRI,GSL104DSMI,GSL104OWRI,GSL104RHEL,GSL105DROD,GSL105OWRI,GSL105RLEW,GSL106DDRE,GSL106OWRI,GSL106RMEH,

In [98]:
print('state file shape: ', state_block_gdf.shape)
print('state file geoid unique: ', state_block_gdf['GEOID20'].nunique())
print('bexport file shape: ', b_export.shape)
print('bexport file geoid unique', b_export['GEOID20'].nunique())

state file shape:  (337039, 34)
state file geoid unique:  337039
bexport file shape:  (337039, 676)
bexport file geoid unique 337039


In [100]:
b_export['GEOID20'].isna().any()

False

In [108]:
b_export.columns[b_export.isna().any() == True]

Index(['G20ATGDSHA', 'G20ATGGWEI', 'G20ATGLWAS', 'G20ATGOWRI', 'G20ATGRHEI',
       'G20AUDDAHM', 'G20AUDGFAI', 'G20AUDLMOO', 'G20AUDOWRI', 'G20AUDRDEF',
       ...
       'GCON15RTHO', 'GCON16DGNI', 'GCON16OWRI', 'GCON16RKEL', 'GCON17DLAM',
       'GCON17OWRI', 'GCON17RPAR', 'GCON18DDOY', 'GCON18OWRI', 'GCON18RNEG'],
      dtype='object', length=662)